# Исследование надежности заемщиков


## Изучение общей информации о данных

In [1]:
import pandas as pd# импортируйте библиотеку pandas

In [2]:
try:
    data = pd.read_csv("/datasets/data.csv")# прочитайте csv-файл
except FileNotFoundError:
    data = pd.read_csv("data.csv")

In [3]:
print(data.head(20))# ваш код здесь

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

In [4]:
data.info()# ваш код здесь

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
print(data.isna().sum())# ваш код здесь

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [6]:
data_group = data.groupby("income_type").total_income.median()
for i in data_group.index:
    data_sort = data.loc[data["income_type"]==i, "total_income"]
    data_sort.fillna(data_group[i], inplace=True)
    data.loc[data.income_type==i, "total_income"] = data_sort

### Обработка аномальных значений

In [7]:
data.days_employed = abs(data.days_employed)# ваш код здесь

In [8]:
print(data.groupby("income_type")["days_employed"].median())# ваш код здесь

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

In [9]:
data.children.unique()# ваш код здесь

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [10]:
data = data[~data.children.isin([-1, 20])]# ваш код здесь

In [11]:
data.children.unique()# ваш код здесь

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [12]:
data_group = data.groupby("income_type").days_employed.median()
for i in data_group.index:
    data.loc[data["income_type"]==i,
             "days_employed"] = data.loc[data["income_type"]==i,
                                         "days_employed"].fillna(data_group[i])
    # ваш код здесь

In [13]:
data.isna().sum()# ваш код здесь

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [14]:
data.total_income = data.total_income.astype(int)# ваш код здесь

### Обработка дубликатов

In [15]:
data.education = data.education.str.lower()# ваш код здесь

In [16]:
data.duplicated().sum()# посчитайте дубликаты

71

In [17]:
data.drop_duplicates(inplace=True)# удалите дубликаты

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [18]:
def categorize_income(row_total):
    category = ["A", "B", "C", "D", "E"]
    values = [1000000, 200000, 50000, 30000, 0]
    for i in range(len(values)):
      if row_total >= values[i]:
        return category[i] # создайте функцию categorize_income()

In [20]:
data["total_income_category"] = data.total_income.apply(categorize_income)# примените функцию методом apply()

In [11]:
data.purpose.unique()# ваш код здесь

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [12]:
def categorize_purpose(row):
    category = ['операции с автомобилем', 'операции с недвижимостью',
                'проведение свадьбы', 'получение образования']
    cat_stem = ['авто', 'жи', 'свадь', 'образов']
    for i in range(len(cat_stem)):
        if cat_stem[i] in row:
            return category[i] # создайте функцию categorize_purpose()

In [13]:
data["purpose_category"] = data.purpose.apply(categorize_purpose)# примените функцию методом apply()

### Исследование данные

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для того чтобы определить наличие или отсутствие зависисмости, необходимо сгуппировать исходный датафрейм `data` по количеству детей в столбце `children` для столбца `debt`. Вычислить количество должников по каждой группе и посчитать конверсию.
Воспользуемся методом `pivot_table`. В качестве функции передадим в стоблец `debt`:
*   `sum`  - для просмотра количества должников,
*   `count` - для подсчета общего количества,
*   `mean` - для вычисления конверсии.

В столбце `respond` для каждой категории вычисляем долю данных от общего количества столбца `count`, чтобы оценить корректность выборки: при малом количестве данных одной или нескольких категорий, оценка конверсии может оказаться не корректной. В конце форматируем таблицу для наглядности.

Для данных действий определим функцию для возможности повторного использования

In [22]:
def convers(df, ind):
    sort_data = df.pivot_table(values="debt", index=ind, 
                           aggfunc=["sum", "count", "mean"])
    sort_data["respond", "debt"] = sort_data["count", "debt"]/sort_data["count", "debt"].sum()
    sort_data.sort_values(("mean","debt"), ascending=False, inplace=True)
    
    prop = {
        "border": "0.5px solid black", 
        "width": "85px", 
        "text-align": "center"   
    }
    percent = {
      ("mean", "debt"):"{:.2%}",
      ("respond", "debt"):"{:.2%}"  
    }    
    sort_data.st = sort_data.style.format(percent)\
    .set_properties(**prop)\
    .bar("mean", color="#8FE8FA")\
    .bar("respond", color="#FFBF4D")
                       
    return sort_data.st   

In [23]:
child_d = convers(df=data, ind="children")
child_d


,sum,count,mean,respond
,debt,debt,debt,debt
children,,,,
4,4,41,9.76%,0.19%
2,194,2052,9.45%,9.62%
1,444,4808,9.23%,22.54%
3,27,330,8.18%,1.55%
0,1063,14091,7.54%,66.06%
5,0,9,0.00%,0.04%


**Вывод:** 


Для людей с количеством детей `5`, `4` и `3` очень маленькая выборка, поэтому основные данные дают люди с `0`, `1` и `2`. Между основными данными разница не очень большая. Исходя из этого можем сделать вывод, что зависимость задолженности от количества детей отсутствует. 



#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Повторим все шаги из предыдущего шага. Группировку датафрейма выполним по столбцам `family_status_id` и `family_status`

In [24]:
status_d = convers(df=data, ind=["family_status_id", "family_status"])
status_d

,,sum,count,mean,respond
,,debt,debt,debt,debt
family_status_id,family_status,,,,
4,Не женат / не замужем,273,2796,9.76%,13.11%
1,гражданский брак,385,4134,9.31%,19.38%
0,женат / замужем,927,12261,7.56%,57.48%
3,в разводе,84,1189,7.06%,5.57%
2,вдовец / вдова,63,951,6.62%,4.46%


**Вывод:** 

По столбцу `respond` можно увидеть, что количество строк для каждой категории достаточно для анализа. По столбцу `convers` наблюдается корреляция между семейным статусом и количеством задолженностей по возврату кредитов: люди не состоящие в официальном браке(категории `не женат/не замужем` и `гражданский брак`) склонны реже платить по долговым обязательствам. Действительно, не имея семьи и обязательств существует риск неверно рассчитать расход собственных средств без возможности опереться на доход супруга/супруги.

Люди из категорий `женат/замужем` являются идеальными заемщиками: один из супругов будет являтся поручителем по кредиту, а совместный доход гарантированно позволит выплачивать долговые обязательства. К тому же, семейные люди реже склонны рисковать и брать кредиты, которые превышают их возможности по выплатам.

Категория `вдова/вдовец` зачастую предполагает поступление средств от государства в пользу этой категории граждан, а так же, возможность использования совместно нажитого имущества единолично. Что объясняет своевременные выплаты.

Заемщики из категории `в разводе` - самые неоднозначные, что можно увидеть из таблице ниже. Основную часть составляют женщины с доходом от 50к до 200к с детьми. Здесь можно предположить, что получаемые алименты способствуют выплатам. Либо, исключая информацию из таблицы ниже, предположить некорректность исходных данных. 


In [25]:
data[(data.family_status=="в разводе")&(data.debt==1)]\
.pivot_table(values=["children"], 
             index=["gender", "total_income_category"], 
             aggfunc = ["count", "sum"])\
             .style.highlight_max("count")\
             .set_properties(**{
                 "border": "0.5px solid black",
                 "width": "100px",
                 "text-align": "center"})

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сгруппируем данные по столбцу `total_income`

In [26]:
income_d = convers(df=data, ind="total_income_category")
income_d

,sum,count,mean,respond
,debt,debt,debt,debt
total_income_category,,,,
E,2,22,9.09%,0.10%
C,1353,15921,8.50%,74.64%
A,2,25,8.00%,0.12%
B,354,5014,7.06%,23.51%
D,21,349,6.02%,1.64%


**Вывод:** 

Для категории с доходом `E` и `A` выборка недостаточна, о чем говорят значения в стоблце `respond`. Ввиду недостаточного количества данных невозможно корректно анализировать зависимость долговых обязательств от дохода

#### Как разные цели кредита влияют на его возврат в срок?

In [27]:
purpose_d = convers(df=data, ind="purpose_category")
purpose_d

,sum,count,mean,respond
,debt,debt,debt,debt
purpose_category,,,,
операции с автомобилем,400,4279,9.35%,20.06%
получение образования,369,3988,9.25%,18.70%
проведение свадьбы,183,2313,7.91%,10.84%
операции с недвижимостью,780,10751,7.26%,50.40%


**Вывод:**

Исходя из столбца `respond` - выборка достаточная для оценки зависимости. На категории `операции с автомобилем` и `получение образования` приходятся основные задолженности. На наиболее многочисленную, по количеству выборки, категорию `операции с недвижимостью` приходится наименьшее количество задолженностей.

## Общий вывод.

В данной работе проведена подготовка данных с последующим их использованием при анализе зависимостей, которые по различным метрикам определяют категории лиц принадлежащие высокой группе риска по своевременным выпалатам долговых обязательств. 

**В первой части** работы данный были откорректированы путем работы с пропущенными и аномальными значениями, удалением дубликатов, изменением типов данных. Так же были добавлены отдельные категории для удобства работы с данными.

**Во второй части работы** на основании подготовленных данных были установлены зависимости наличия задолженностей по кредитам и семейного статуса, а так же влияние целей кредита на задолженности. Влияние количества детей и суммы дохода на наличие задолженностей не установлено по причине малой выборки данных для анализа.

Для таблицы влияния количества детей недосточно данных для категорий `4`, `3` и `5`. Для таблицы влияния суммы дохода данных недостает в категория `E`, `A` и`D`. В случае продолжения анализа, выводы будет ошибочны.

In [28]:
from IPython.display import display_html
income_d = income_d.set_table_attributes("style='display:inline'")
child_d = child_d.set_table_attributes("style='display:inline'")
display_html(child_d._repr_html_()+income_d._repr_html_(), raw=True)

,sum,count,mean,respond
,debt,debt,debt,debt
children,,,,
4,4,41,9.76%,0.19%
2,194,2052,9.45%,9.62%
1,444,4808,9.23%,22.54%
3,27,330,8.18%,1.55%
0,1063,14091,7.54%,66.06%
5,0,9,0.00%,0.04%
,sum,count,mean,respond
,debt,debt,debt,debt


При анализы влияния целей кредита и семейного статуса на выплату по долговым обязательствам наименее предпочтительные для кредитора категории: `операции с автомобилем` и `получение образования`, `не женат/не замужем` и `гражданский брак`.

In [29]:
status_d = status_d.set_table_attributes("style='display:inline'")
purpose_d = purpose_d.set_table_attributes("style='display:inline'")
display_html(purpose_d._repr_html_()+status_d._repr_html_(), raw=True)

sum count mean respond debt debt debt debt purpose_category 
 
 операции с автомобилем 
 400 
 4279 
 9.35% 
 20.06% 
 
 
 получение образования 
 369 
 3988 
 9.25% 
 18.70% 
 
 
 проведение свадьбы 
 183 
 2313 
 7.91% 
 10.84% 
 
 
 операции с недвижимостью 
 780 
 10751 
 7.26% 
 50.40% 
 
 sum count mean respond debt debt debt debt family_status_id family_status 
 
 4 
 Не женат / не замужем 
 273 
 2796 
 9.76% 
 13.11% 
 
 
 1 
 гражданский брак 
 385 
 4134 
 9.31% 
 19.38% 
 
 
 0 
 женат / замужем 
 927 
 12261 
 7.56% 
 57.48% 
 
 
 3 
 в разводе 
 84 
 1189 
 7.06% 
 5.57% 
 
 
 2 
 вдовец / вдова 
 63 
 951 
 6.62% 
 4.46%

Наиболее предпочтительные клиенты вдовцы/вдовы с целью получения кредита для совершения операций с недвижимостью. И наборот, менее надеждные заемщики - не женатые/не замужние, собирающие потратить деньги на автомобиль.